In [ ]:
import gdown
gdown.download('https://bit.ly/3GisL6J','ns_book4.csv',quiet=False)

import pandas as pd
ns_book4 = pd.read_csv('ns_book4.csv',low_memory=False)
ns_book4.head() # NaN이 좀 있는데 이걸 이제 해결해볼거임
ns_book4.info() # 전채 행 개수, 열개수, 열 이름, 데이터 타입, 메모리 사용량, NaN개수 알 수 있음
#ns_book4.info(memory_usage='deep') : 정확한 메모리 사용량이 나옴

# 누락된 값 처리하기
ns_book4.isna().sum() # 각 행이 NaN인지 불리언 배열 반환 + True의 값의 합 -> 누적된 NaN의 개수
ns_book4.notna().sum() # 누락되지 않은 값(isna의 반대)

# 누락된 값으로 표시하기 : None,np.nan 활용
ns_book4.loc[0,'도서권수'] = None # 첫행,도서권수 열을 None으로 바꿈
print(ns_book4['도서권수'].isna().sum()) # 원래 도서권수 누락이 0이었는데 1로 이제 출력함
print(ns_book4.head(2)) # 첫 행의 도서권수가 NaN으로 바뀌고 NaN이 특별한 실수라서 1행의 도서권수도 1.0으로 바뀜
print()

ns_book4.loc[0,'도서권수'] = 1 # 첫행,도서권수 열을 1로 바꿈
ns_book4 = ns_book4.astype({'도서권수': 'int32', '대출건수': 'int32'}) #type을 int형으로 바꿔줌
print(ns_book4.head(2))

# 문자열의 경우 np.nan을 사용해야한다
import numpy as np
ns_book4.loc[0,'부가기호'] = np.nan
print(ns_book4.head(2))

# 누락된 값 바꾸기(1) : loc, fillna()
set_isbn_na_rows = ns_book4['세트 ISBN'].isna() # 누락된 값을 찾아 불리언 배열로 변환
ns_book4.loc[set_isbn_na_rows, '세트 ISBN'] = '' # 누락된 값을 빈 문자열로 변환
ns_book4['세트 ISBN'].isna().sum() # 누락된 개수가 이제 0개라는 것을 확인 가능

ns_book4.fillna('없음').isna().sum() # 모든 NaN을 없음으로 변환
ns_book4.fillna({'부가기호':'없음'}).isna().sum() # 특정 열만 없음으로 바꾸기 위해서는 딕셔너리 형태로 쓴다.

# 누락된 값 바꾸기(2) : replace()
# 1. 바꾸려는 값이 1개일 때 replace(원래 값, 바꿀 값)
ns_book4.replace(np.nan,'없음').isna().sum()
# 2. 여러개 일 때 replace([원래 값1,원래 값2], [새로운 값1,새로운 값2])
ns_book4.replace([np.nan,'2021'],['없음','21']).head(2)
# 3. 열 마다 다른 값으로 바꿀 때 replace({열 이름, 원래 값}, 새로운 값) or replace(열 이름{원래 값1 : 새로운 값1})
ns_book4.replace({'부가기호': np.nan},'없음').head(2)
ns_book4.replace({'부가기호': {np.nan:'없음'},'발행년도':{'2021':'21'}}).head(2)

# 정규 표현식 : 숫자 = \d,숫자를 제외한 나머지 = \D, \d\d = \d{2}, regex = True로 놓기 \1 = 첫번째 그룹 사용
ns_book4.replace({'발행년도': {r'\d\d(\d\d)': r'\1'}},regex=True)[100:102] # 2018 -> 18로 변함
# 문자찾기 : 마침표(.) \s = 공백문자, . = 어떤 문자에도 대응, * = 모두,
ns_book4.replace({'저자': {r'(.*)\s\(지은이\)(.*)\s\(옮긴이\)':r'\1\2'}, '발행년도': {r'\d{2}(\d{2})': r'\1'}},regex = True)[100:102] # 로런스 인그래시아 (지은이), 안기순 (옮긴이)에서
# 로런스 인그래시아, 안기순으로 옮김, 2018 -> 18 등으로 연도 바꿈

# 잘못된 값 바꾸기
ns_book4['발행년도'].str.contains('1988').sum() # 407
invalid_number = ns_book4['발행년도'].str.contains('\D', na=True) # 숫자 이외의 값 -> True로 놓은 배열
print(invalid_number.sum()) # 1777개나 숫자가 아닌게 있다.
ns_book4[invalid_number].head()

ns_book5 = ns_book4.replace({'발행년도':r'.*(\d{4}).*'},r'\1',regex=True) # 연도 숫자 4개를 그룹으로 묶어 \1로 참조. 그 뒤에 어떤 문자여도 매칭하기 위해 .*사용
ns_book5[invalid_number].head()

unkown_year = ns_book5['발행년도'].str.contains('\D', na=True) # 발행년도가 숫자외의 자료형이면 True인 배열
print(unkown_year.sum()) # 67개로 줄었다.
ns_book5[unkown_year].head() # 이제 남은건 NaN이거나 4자리가 아니거나이다.

ns_book5.loc[unkown_year, '발행년도'] = '-1' # 임의로 NaN이거나 4자리가 아닌 값을 -1로 바꾼다.
ns_book5 = ns_book5.astype({'발행년도': 'int32'}) # 자료형을 int32로 바꿈

ns_book5['발행년도'].gt(4000).sum() # 4000보다 큰 값의 개수 : 131, (ns_book5['발행년도']>4000).sum()과 같다.

dangun_yy_rows = ns_book5['발행년도'].gt(4000) # 4000보다 큰 값을 넣는다.
ns_book5.loc[dangun_yy_rows, '발행년도'] = ns_book5.loc[dangun_yy_rows, '발행년도'] - 2333 # 찾은 값에서 2333을 뺌.

dangun_year = ns_book5['발행년도'].gt(4000) # 다시 찾는다.
print(dangun_year.sum()) # 13개나 있다.
ns_book5[dangun_year].head(2)

ns_book5.loc[dangun_year, '발행년도'] = -1 # -1로 변환

old_books = ns_book5['발행년도'].gt(0) & ns_book5['발행년도'].lt(1900) # 0<x<1900인 값 찾기
ns_book5[old_books]

ns_book5.loc[old_books, '발행년도'] = -1 # 여전히 잘못된 값을 -1로 변환
ns_book5['발행년도'].eq(-1).sum() #-1과 같은 값 = 86개

# 누락된 정보 채우기
# 및의 4개의 열은 누락되면 안된다.
na_rows = ns_book5['도서명'].isna() | ns_book5['저자'].isna() | ns_book5['출판사'].isna() | ns_book5['발행년도'].eq(-1)
print(na_rows.sum()) # 5268개나 누락...
ns_book5[na_rows].head(2)

import requests
from bs4 import BeautifulSoup
# 책 제목 채우기
def get_book_title(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 클래스 이름이 'gd_name'인 a 태그의 텍스트를 가져옵니다.
    title = soup.find('a', attrs={'class':'gd_name'}).get_text()
    return title
get_book_title(9791191266054)

import re # 정규표현식 모듈
# 저자,출판사,발행 연도 추출 후 반환
def get_book_info(row):
    title = row['도서명']
    author = row['저자']
    pub = row['출판사']
    year = row['발행년도']
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(row['ISBN']))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    try:
        if pd.isna(title):
            # 클래스 이름이 'gd_name'인 a 태그의 텍스트를 가져옵니다.
            title = soup.find('a', attrs={'class':'gd_name'}) \
                    .get_text()
    except AttributeError:
        pass

    try:
        if pd.isna(author):
            # 클래스 이름이 'info_auth'인 span 태그 아래 a 태그의 텍스트를 가져옵니다.
            authors = soup.find('span', attrs={'class':'info_auth'}) \
                          .find_all('a')
            author_list = [auth.get_text() for auth in authors]
            author = ','.join(author_list)
    except AttributeError:
        pass

    try:
        if pd.isna(pub):
            # 클래스 이름이 'info_auth'인 span 태그 아래 a 태그의 텍스트를 가져옵니다.
            pub = soup.find('span', attrs={'class':'info_pub'}) \
                      .find('a') \
                      .get_text()
    except AttributeError:
        pass

    try:
        if year == -1:
            # 클래스 이름이 'info_date'인 span 태그 아래 텍스트를 가져옵니다.
            year_str = soup.find('span', attrs={'class':'info_date'}) \
                           .get_text()
            # 정규식으로 찾은 값 중에 첫 번째 것만 사용합니다.
            year = re.findall(r'\d{4}', year_str)[0]
    except AttributeError:
        pass

    return title, author, pub, year

updated_sample = ns_book5[na_rows].head(2).apply(get_book_info,axis=1, result_type ='expand') # ns_book5[na_rows]의 2개에 대해 get_book_info 적용
updated_sample

# 5268개를 모두 적용하기에는 시간이 걸리므로 미리 만들어놓은 ns_book5_update.csv로 보자.
# ns_book5_update = ns_book5[na_rows].apply(get_book_info,axis=1, result_type ='expand')
# ns_book5_update.columns = ['도서명','저자','출판사','발행년도']
# ns_book5_update.head()
gdown.download('http://bit.ly/3UJZiHw', 'ns_book5_update.csv', quiet=False)

ns_book5_update = pd.read_csv('ns_book5_update.csv', index_col=0) # 데이터프레임으로 변환
ns_book5_update.head()

ns_book5.update(ns_book5_update) # ns_book5_update.csv로 ns_book5_update배열에 업데이트한다.

na_rows = ns_book5['도서명'].isna() | ns_book5['저자'].isna() \
          | ns_book5['출판사'].isna() | ns_book5['발행년도'].eq(-1) # NaN인 열을 -1로 변환
print(na_rows.sum()) # 4615

ns_book5 = ns_book5.astype({'발행년도': 'int32'}) # 발행년도 타입 =int형으로 변환
ns_book6 = ns_book5.dropna(subset=['도서명','저자','출판사']) # 도서명,저자,출판사 열이 NaN인 행 없애기
ns_book6 = ns_book6[ns_book6['발행년도'] != -1] # 발행년도가 -1이 아닌것만 가지기
ns_book6.head()

ns_book6.to_csv('ns_book6.csv', index=False)

# 지금까지의 과정 정리한 함수
def data_fixing(ns_book4):
    """
    잘못된 값을 수정하거나 NaN 값을 채우는 함수

    :param ns_book4: data_cleaning() 함수에서 전처리된 데이터프레임
    """
    # 도서권수와 대출건수를 int32로 바꿉니다.
    ns_book4 = ns_book4.astype({'도서권수':'int32', '대출건수': 'int32'})
    # NaN인 세트 ISBN을 빈문자열로 바꿉니다.
    set_isbn_na_rows = ns_book4['세트 ISBN'].isna()
    ns_book4.loc[set_isbn_na_rows, '세트 ISBN'] = ''

    # 발행년도 열에서 연도 네 자리를 추출하여 대체합니다. 나머지 발행년도는 -1로 바꿉니다.
    ns_book5 = ns_book4.replace({'발행년도':'.*(\d{4}).*'}, r'\1', regex=True)
    unkown_year = ns_book5['발행년도'].str.contains('\D', na=True)
    ns_book5.loc[unkown_year, '발행년도'] = '-1'

    # 발행년도를 int32로 바꿉니다.
    ns_book5 = ns_book5.astype({'발행년도': 'int32'})
    # 4000년 이상인 경우 2333년을 뺍니다.
    dangun_yy_rows = ns_book5['발행년도'].gt(4000)
    ns_book5.loc[dangun_yy_rows, '발행년도'] = ns_book5.loc[dangun_yy_rows, '발행년도'] - 2333
    # 여전히 4000년 이상인 경우 -1로 바꿉니다.
    dangun_year = ns_book5['발행년도'].gt(4000)
    ns_book5.loc[dangun_year, '발행년도'] = -1
    # 0~1900년 사이의 발행년도는 -1로 바꿉니다.
    old_books = ns_book5['발행년도'].gt(0) & ns_book5['발행년도'].lt(1900)
    ns_book5.loc[old_books, '발행년도'] = -1

    # 도서명, 저자, 출판사가 NaN이거나 발행년도가 -1인 행을 찾습니다.
    na_rows = ns_book5['도서명'].isna() | ns_book5['저자'].isna() \
              | ns_book5['출판사'].isna() | ns_book5['발행년도'].eq(-1)
    # 교보문고 도서 상세 페이지에서 누락된 정보를 채웁니다.
    updated_sample = ns_book5[na_rows].apply(get_book_info,
        axis=1, result_type ='expand')
    updated_sample.columns = ['도서명','저자','출판사','발행년도']
    ns_book5.update(updated_sample)

    # 도서명, 저자, 출판사가 NaN이거나 발행년도가 -1인 행을 삭제합니다.
    ns_book6 = ns_book5.dropna(subset=['도서명','저자','출판사'])
    ns_book6 = ns_book6[ns_book6['발행년도'] != -1]

    return ns_book6

Downloading...
From: https://bit.ly/3GisL6J
To: /content/ns_book4.csv
100%|██████████| 55.5M/55.5M [00:00<00:00, 230MB/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384591 entries, 0 to 384590
Data columns (total 13 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   번호       384591 non-null  int64  
 1   도서명      384188 non-null  object 
 2   저자       384393 non-null  object 
 3   출판사      379950 non-null  object 
 4   발행년도     384577 non-null  object 
 5   ISBN     384591 non-null  object 
 6   세트 ISBN  56576 non-null   object 
 7   부가기호     310386 non-null  object 
 8   권        63378 non-null   object 
 9   주제분류번호   364727 non-null  object 
 10  도서권수     384591 non-null  int64  
 11  대출건수     384591 non-null  float64
 12  등록일자     384591 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 38.1+ MB
1
   번호           도서명      저자   출판사  발행년도           ISBN 세트 ISBN 부가기호    권  \
0   1       인공지능과 흙  김동훈 지음   민음사  2021  9788937444319     NaN  NaN  NaN   
1   2  가짜 행복 권하는 사회  김태형 지음  갈매나무  2021  9791190123969     NaN  NaN  NaN   

  주제분류번호 

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
78,79,아산 정주영 레거시,김화진,NaN,2021,9788952129529,,NaN,NaN,325,1,1,2021-03-15
265,278,골목의 시간을 그리다,정명섭.김효찬 지음,NaN,2021,9791191266054,,NaN,NaN,NaN,1,0,2021-03-12
